In [1]:
from surprise import Dataset, Reader
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import accuracy
import pandas as pd
from surprise.model_selection import cross_validate
import os
import pickle

In [24]:

def read_ratings(ratings_csv: str, data_dir: str = "/home/antoine/PROJET_MLOPS_RECO_MOVIES/data/raw/silver") -> pd.DataFrame:
    """
    Lit le fichier CSV contenant les évaluations des films.

    :param ratings_csv: Nom du fichier CSV contenant les évaluations.
    :param data_dir: Répertoire où se trouve le fichier CSV.
    :return: DataFrame contenant les évaluations.
    """
    try:
        filepath = os.path.join(data_dir, ratings_csv)
        df = pd.read_csv(filepath)
        print("Dataset ratings chargé")
        return df
    except FileNotFoundError:
        print(f"Erreur: Le fichier {filepath} n'a pas été trouvé.")
        return None


def train_model(df: pd.DataFrame, n_factors: int = 150, n_epochs: int = 30, lr_all: float = 0.01, reg_all: float = 0.05) -> SVD:
    """Entraîne le modèle de recommandation sur les données fournies."""
    # Diviser les données en ensembles d'entraînement et de test
    reader = Reader(rating_scale=(0.5, 5))

    data = Dataset.load_from_df(df[['userid', 'movieid', 'rating']], reader=reader)

    # Extraire le Trainset
    trainset = data.build_full_trainset()

    model = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_all=reg_all)

    # Entraîner le modèle
    model.fit(trainset)

    print("Début de la cross-validation")

    # Effectuer la validation croisée sur le Trainset
    cv_results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, return_train_measures=True)

    # Afficher les résultats
    mean_rmse = cv_results['test_rmse'].mean()
    print("Moyenne des RMSE :", mean_rmse)

    return model, mean_rmse, reader


def save_model(model: SVD, filepath: str, version: str, reader: Reader) -> None:
    """Sauvegarde le modèle entraîné dans un fichier."""
    os.makedirs(os.path.dirname(filepath), exist_ok=True)

    # Modifier le nom du fichier pour inclure la version
    base, ext = os.path.splitext(filepath)
    versioned_filepath = f"{base}_{version}{ext}"

    # Sauvegarder le modèle et le contexte (reader)
    with open(versioned_filepath, 'wb') as file:
        pickle.dump({'model': model, 'reader': reader}, file)
        print(f'Modèle sauvegardé sous {versioned_filepath}')


In [25]:
ratings = read_ratings('processed_ratings.csv')

ratings.head()

Dataset ratings chargé


,userid,movieid,rating,timestamp,bayesian_mean
0,1,2,3.5,1112486027,3.21
1,1,29,3.5,1112484676,3.89
2,1,32,3.5,1112484819,3.89
3,1,47,3.5,1112484727,4.04
4,1,50,3.5,1112484580,4.32


In [26]:
svd, mean_rmse, reader = train_model(ratings)

Début de la cross-validation
Moyenne des RMSE : 0.7845606211457105


In [27]:
save_model(svd, '/home/antoine/PROJET_MLOPS_RECO_MOVIES/data/models/svd_model.pkl', 'v1', reader=reader)

Modèle sauvegardé sous /home/antoine/PROJET_MLOPS_RECO_MOVIES/data/models/svd_model_v1.pkl


In [ ]:
def load_model(filepath: str, version: str):
    """Charge le modèle entraîné depuis un fichier."""
    base, ext = os.path.splitext(filepath)
    versioned_filepath = f"{base}_{version}{ext}"

    with open(versioned_filepath, 'rb') as file:
        model_data = pickle.load(file)
        model = model_data['model']
        reader = model_data['reader']
        print(f'Modèle chargé depuis {versioned_filepath}')
        return model, reader

In [28]:
import pandas as pd
from surprise import SVD, Dataset

def recommend_movies(model: SVD, user_id: int, df: pd.DataFrame, top_n: int = 10) -> list:
    """
    Recommande des films à un utilisateur en fonction de ses évaluations prédites.

    :param model: Le modèle SVD entraîné.
    :param user_id: L'ID de l'utilisateur pour lequel on veut des recommandations.
    :param df: DataFrame contenant les données des films et les évaluations.
    :param top_n: Le nombre de films à recommander.
    :return: Une liste des top_n films recommandés.
    """
    # 1. Obtenir la liste des films que l'utilisateur a déjà évalués
    movies_already_rated = df[df['userid'] == user_id]['movieid'].unique()

    # 2. Créer une liste de tous les films possibles
    all_movie_ids = df['movieid'].unique()

    # 3. Filtrer les films que l'utilisateur n'a pas encore évalués
    movies_to_predict = [movie_id for movie_id in all_movie_ids if movie_id not in movies_already_rated]

    # 4. Faire des prédictions pour chaque film non évalué
    predictions = []
    for movie_id in movies_to_predict:
        predictions.append((movie_id, model.predict(user_id, movie_id).est))

    # 5. Trier les prédictions par ordre décroissant d'évaluation prédite
    predictions.sort(key=lambda x: x[1], reverse=True)

    # 6. Retourner les top_n films
    top_recommendations = [movie_id for movie_id, _ in predictions[:top_n]]
    return top_recommendations


In [29]:
movies = pd.read_csv('/home/antoine/PROJET_MLOPS_RECO_MOVIES/data/raw/silver/processed_movies.csv')
movie_titles = dict(zip(movies["movieid"], movies["title"]))

In [30]:

result = recommend_movies(svd, 1, ratings)

for movie_id in result:
    print(movie_titles[movie_id])

The War at Home
Very Potter Musical, A
Zero Motivation (Efes beyahasei enosh)
Crooks in Clover (a.k.a. Monsieur Gangster) (Les tontons flingueurs)
Welfare
Tito and Me (Tito i ja)
Matrix, The
For Neda
Rewind This!
Look of Silence, The


In [31]:
result2 = recommend_movies(svd, 501, ratings)

for movie_id in result2:
    print(movie_titles[movie_id])

The Salt of the Earth
Tito and Me (Tito i ja)
Welfare
Zero Motivation (Efes beyahasei enosh)
Look of Silence, The
Lake, A (Un lac)
Dirties, The
Shawshank Redemption, The
Strangers in Good Company
Crooks in Clover (a.k.a. Monsieur Gangster) (Les tontons flingueurs)
